# Demand response modeling tutorial
Welcome fellow oemof friends! :-)

This tutorial will guide you through 3 examples for using the SinkDSM component in its current implementation (oemof v0.4.4):
1. A simple example building everything from the sratch and for one modeling approach (the new "DLR" one)
2. Introducing some functions and comparing approaches against each other (for the same very simple example).
3. Setting up an exemplary investment model with the same functions.

> **_Sections which can be skipped since they are important for processing but not the actual SinkDSM implementation in oemof.solph are marked the same as this line._** 

# Package Imports and plot settings

> __*Feel free to skip that chapter*__

Imports:
* Standard imports
* Import the different implementations for demand response components
* Import module `plotting.py` for extracting results and visualization

Plot settings:<br>
* Register matplotlib converters.
* Adjust matplotlib standard settings for graphs.
* Create a directory to store graphs (if it doesn't already exist).

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters

import oemof.solph as solph
from oemof.network.network import Node

# Import module for plotting (results handling)
import plotting as plt_dsm

In [ ]:
# Determine matplotlib settings
register_matplotlib_converters()

SMALL_SIZE = 11
MEDIUM_SIZE = 14
BIGGER_SIZE = 15

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
# Make folder for graphics
plt_dsm.make_directory('graphics')

# DSM parameter settings
Define major **parameters concerning demand response modelling**
* *aproaches*: List of the approaches used for demand response modelling.

Determine **costs for demand response**:
* *cost_dsm*: Overall variable costs for demand response which have to be splitted up to up and downwards shifts
* *cost_dsm_up*: Costs for upwards shifts (_defaults to have of the overall costs_)
* *cost_dsm_down*: Costs for downwards shifts (_defaults to have of the overall costs_)

In [ ]:
# Parameters focussing on demand response modelling
approaches = ['oemof', 'DIW', 'DLR']

# Cost is split half on upwards and downwards shift; shedding gets high costs
cost_dsm = 0.1
cost_dsm_up = cost_dsm/2
cost_dsm_down_shift = cost_dsm/2
cost_dsm_down_shed = 1000 * cost_dsm

# Toy energy system model
For this tutorial, a **toy energy system** is set up including:
- Coal PP
- Wind PP
- DSM Sink
- shortage
- excess

**Rules for DSM parametrization**:

The following rules apply for parameters which are not part of every modelling approach:<br>
* shift (resp. interference) times: These will be defined half of the delay time and symmetrical in the first place.

**_Please change to your solver of choice!_**

In [ ]:
solver = 'cbc'
datetimeindex = pd.date_range(start='1/1/2013', periods=9, freq='H')

## Create generation and load data set for toy model: Base data set
* A basic data set for the toy model is defined in the following.
* To analyze different behaviour of the modelling approaches, this data set is modified below.

In [ ]:
timesteps = 48

# base data set
demand = [100] * timesteps
capup = [100] * timesteps
capdo = [100] * timesteps
wind = [100] * timesteps
 
base = [demand, wind, capup, capdo]
df_base = pd.DataFrame(list(zip(*base)))
df_base.rename(columns={0:'demand_el',1:'wind', 2:'Cap_up', 3:'Cap_do'}, inplace=True)
df_base['timestamp'] = pd.date_range(start='1/1/2013', periods=timesteps, freq='H')
df_base.set_index('timestamp', drop=True, inplace=True)

In [ ]:
plt_dsm.plot_case(data=df_base, case="base data set")
plt_dsm.plot_case_residual(data=df_base, case="base data set")

## Introduce a demand variation

In [ ]:
case = 'simple'

# Base data set
df_data = df_base.copy()
demand = [100] * timesteps

# Manipulate demand
demand[1:2] = [150]
demand[5:6] = [50]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plt_dsm.plot_case(data=df_data, case=case)
plt_dsm.plot_case_residual(data=df_data, case=case)

# Example 1: Define an energy system, add demand response to it and evaluate the resuls

## Step 1: Create an energy system

In [ ]:
# Create Energy System
es = solph.EnergySystem(timeindex=datetimeindex)

Node.registry = es

# Create Buses
b_coal_1 = solph.Bus(label='bus_coal_1')
b_elec = solph.Bus(label='bus_elec')

# Create Sources
s_coal_p1 = solph.Source(label='source_coal_p1',
                         outputs={
                             b_coal_1: solph.Flow(
                                 nominal_value=10000,
                                 variable_costs=13)}
                        ) 

s_wind = solph.Source(label='wind',
                      outputs={
                          b_elec: solph.Flow(
                              fix=df_data['wind'][datetimeindex],
                              nominal_value=1)}
                      )

# Create Transformers
cfp_1 = solph.Transformer(label='pp_coal_1',
                          inputs={b_coal_1: solph.Flow()},
                          outputs={
                                b_elec: solph.Flow(
                                    variable_costs=0)},
                          conversion_factors={b_elec: 0.4}
                         )

# Backup excess / shortage
excess = solph.Sink(label='excess_el',
                    inputs={b_elec: solph.Flow(variable_costs=1)}
                    )

s_shortage_el = solph.Source(label='shortage_el',
                             outputs={
                                 b_elec: solph.Flow(
                                     variable_costs=200)}
                             )

## Step 2: Add demand response to the energy system

In [ ]:
demand_dsm = solph.custom.SinkDSM(
    label = 'demand_dsm',
    demand = df_data['demand_el'][datetimeindex],
    capacity_up = df_data['Cap_up'][datetimeindex],
    capacity_down = df_data['Cap_do'][datetimeindex],
    approach = "DLR", 
    inputs = {b_elec: solph.Flow(variable_costs=0)},
    delay_time = 4,  # TODO: Rename! compensation_time
    cost_dsm_up = cost_dsm_up,
    cost_dsm_down_shift = cost_dsm_down_shift,
    shed_eligibility = False,
    max_demand = 1,
    max_capacity_up = 1,
    max_capacity_down = 1,
    shift_time = 2,
)

## Step 3: Solve and inspect the results

In [ ]:
model = solph.Model(es)

# Solve Model
model.solve(solver='cbc', solve_kwargs={'tee': False})

es.results['main'] = solph.processing.results(model)
es.results['meta'] = solph.processing.meta_results(model)

## Extract model results and plot the model
A function is defined here to extract results from the model and plot the model results.

In [ ]:
def extract_and_plot_results(data, datetimeindex, model, es, approach, case, invest, **kwargs):
    """Extract model results and plot them. """
    
    directory = "./"
    project = 'demand_shift_' + approach + '_' + case
    save = kwargs.get('save', False)
    figure_size = kwargs.get('figsize', (15,10))
    ax1_ylim = kwargs.get('ax1_ylim', [-10, 250])
    ax2_ylim = kwargs.get('ax2_ylim', [-110, 150])
    include_generators = kwargs.get('include_generators', False)
    
    max_demand = data['demand_el'][datetimeindex].max()
    max_capacity_down = data['Cap_do'][datetimeindex].max()
    max_capacity_up = data['Cap_up'][datetimeindex].max()
    
    # Export data
    if invest:
        df_gesamt, dsm_invest = plt_dsm.extract_results(model, approach, 
                                                        invest=invest)     
    else:
        df_gesamt = plt_dsm.extract_results(model, approach, 
                                            invest=invest,
                                            normalized=kwargs.get("normalized", True),
                                            max_demand=max_demand,
                                            max_capacity_down=max_capacity_down,
                                            max_capacity_up=max_capacity_up)
    
    #  Plot Results
    plt_dsm.plot_dsm(df_gesamt,
            directory,
            project,
            days=2,
            show=True,
            figsize=figure_size,
            ax1_ylim=ax1_ylim,
            ax2_ylim=ax2_ylim,
            include_generators=include_generators,
            approach=approach,
            save=save)

    if invest: 
        return df_gesamt, dsm_invest
    else:
        return df_gesamt

In [ ]:
df_gesamt = extract_and_plot_results(data=df_data,
                                     datetimeindex=datetimeindex,
                                     model=model,
                                     es=es,
                                     approach="DLR", 
                                     case="simple",
                                     invest=False,
                                     normalized=False)

In [ ]:
df_gesamt

In [ ]:
# Option to inspect the results and constraints for the SinkDSM (uncomment line below if you are not afraid of cryptic pyomo output ;-))
#model.SinkDSMDLRBlock.pprint()

# Example 2: Compare approaches and introduce functions for reusability
* The following section first introduces some function definitions enabling to evaluate the SinkDSM for arbitrary load and generation patterns.
* Then, the functions are called in a for loop iterating over all of the approaches.
* This method has been applied in a very similar manner to compile the INREC 2020 results, see
    * Slides: https://github.com/jokochems/DR_modeling_oemof/blob/master/Kochems_Demand_Response_INREC.pdf and
    * Code: https://github.com/jokochems/DR_modeling_oemof/blob/master/INREC_examples/DSM-Modelling-Example.ipynb

## Create and solve the model
A function is defined here for setting up a toy energy system.

In [ ]:
def create_energy_system(data, datetimeindex, invest, approach, 
                         delay_time, shed_time, cost_dsm_up, cost_dsm_down_shift, cost_dsm_down_shed,
                         efficiency, shed_eligibility, shift_eligibility,
                         **kwargs):
    """Set up an example energy system. """
    
    # Control generation units
    nom_cap_coal = kwargs.get('nom_cap_coal', 10000)
    
    # Create Energy System
    es = solph.EnergySystem(timeindex=datetimeindex) 
    
    Node.registry = es

    # Create Buses
    b_coal_1 = solph.Bus(label='bus_coal_1')
    b_elec = solph.Bus(label='bus_elec')

    # Create Sources
    s_coal_p1 = solph.Source(label='source_coal_p1',
                             outputs={
                                 b_coal_1: solph.Flow(
                                     nominal_value=nom_cap_coal,
                                     variable_costs=13)}
                            ) 

    s_wind = solph.Source(label='wind',
                          outputs={
                              b_elec: solph.Flow(
                                  fix=data['wind'][datetimeindex],
                                  nominal_value=1)}
                          )

    # Create Transformers
    cfp_1 = solph.Transformer(label='pp_coal_1',
                              inputs={b_coal_1: solph.Flow()},
                              outputs={
                                    b_elec: solph.Flow(
                                        variable_costs=0)},
                              conversion_factors={b_elec: 0.4}
                             )

    # Backup excess / shortage
    excess = solph.Sink(label='excess_el',
                        inputs={b_elec: solph.Flow(variable_costs=1)}
                        )

    s_shortage_el = solph.Source(label='shortage_el',
                                 outputs={
                                     b_elec: solph.Flow(
                                         variable_costs=200)}
                                 )
    
    # Add DSM units
    add_dsm_unit(b_elec, data, datetimeindex, invest, approach, delay_time, shed_time, cost_dsm_up, cost_dsm_down_shift, 
                 cost_dsm_down_shed, efficiency, shed_eligibility, shift_eligibility, **kwargs)
    
    return es

A separate function holds the (general) DSM representation which is added to the energy system.

In [ ]:
def add_dsm_unit(b_elec, data, datetimeindex, invest, approach, delay_time, shed_time, cost_dsm_up, cost_dsm_down_shift, 
                 cost_dsm_down_shed, efficiency, shed_eligibility, shift_eligibility, **kwargs): 
    """Add dsm units to the energy system. """
    
    # DSM parameters
    recovery_time_shift = kwargs.get('recovery_time_shift', None)
    recovery_time_shed = kwargs.get('recovery_time_shed', 24)
    shift_time = kwargs.get('shift_time', delay_time/2)  
    addition = kwargs.get('addition', True)
    fixes = kwargs.get('fixes', True)
    ActivateYearLimit = kwargs.get('ActivateYearLimit', False)
    ActivateDayLimit = kwargs.get('ActivateDayLimit', False)
    
    # Investment modeling
    max_demand = kwargs.get('max_demand', None)
    max_capacity_down = kwargs.get('max_capacity_down', None)
    max_capacity_up = kwargs.get('max_capacity_up', None)
    flex_share_down = kwargs.get('flex_share_down', None)
    flex_share_up = kwargs.get('flex_share_up', None)
    ep_costs = kwargs.get('ep_costs', 1000)
    minimum = kwargs.get('minimum', 0)
    maximum = kwargs.get('maximum', 200)
    existing = kwargs.get('existing', 0)
    
    # Define kwargs that differ dependent on approach chosen   
    if invest:
        max_demand = None
        max_capacity_down = None
        max_capacity_up = None
    else:
        max_demand = data['demand_el'][datetimeindex].max()
        max_capacity_down = data['Cap_do'][datetimeindex].max()
        max_capacity_up = data['Cap_up'][datetimeindex].max()

    # Define kwargs that are identical for all DSM units
    kwargs_all = {'label': 'demand_dsm',
                  'inputs': {b_elec: solph.Flow(variable_costs=0)},
                  'demand': data['demand_el'][datetimeindex],
                  'capacity_up': data['Cap_up'][datetimeindex],
                  'capacity_down': data['Cap_do'][datetimeindex],
                  'delay_time': delay_time,
                  'shed_time': shed_time,
                  'recovery_time_shift': recovery_time_shift,
                  'recovery_time_shed': recovery_time_shed,
                  'cost_dsm_up': cost_dsm_up,
                  'cost_dsm_down_shift': cost_dsm_down_shift,
                  'cost_dsm_down_shed': cost_dsm_down_shed,
                  'efficiency': efficiency,
                  'shed_eligibility': shed_eligibility,
                  'shift_eligibility': shift_eligibility,
                  'max_demand': max_demand,
                  'max_capacity_down': max_capacity_down,
                  'max_capacity_up': max_capacity_up,
                  'flex_share_down': flex_share_down,
                  'flex_share_up': flex_share_up,
                  'shift_time': shift_time}
    
    # Determine recovery / max activations / shed time dependent on each other
    if recovery_time_shift is not None:
        n_yearLimit_shift = kwargs.get('n_yearLimit_shift', 
                                       len(data.loc[datetimeindex,:]) // (delay_time + recovery_time_shift))
    else:
        n_yearLimit_shift = kwargs.get('n_yearLimit_shift', 
                                       len(data.loc[datetimeindex,:]) // delay_time)
    
    if recovery_time_shed is not None:
        n_yearLimit_shed = kwargs.get('n_yearLimit_shed', 
                                      len(data.loc[datetimeindex,:]) // (shed_time + recovery_time_shed))
    else:
        n_yearLimit_shed = kwargs.get('n_yearLimit_shed', 
                                      len(data.loc[datetimeindex,:]) // shed_time)
    t_dayLimit = kwargs.get('t_dayLimit', 0) 
    
    # Use a dict to store the keywords that differ by approach
    kwargs_dict = {
        'oemof': {'approach': approach,
                  'shift_interval': kwargs.get('shift_interval', 24)},

        'DIW': {'approach': approach},
        
        'DLR': {'approach': approach,
                'ActivateYearLimit': ActivateYearLimit,
                'ActivateDayLimit': ActivateDayLimit,
                'n_yearLimit_shift': n_yearLimit_shift,
                'n_yearLimit_shed': n_yearLimit_shed,
                't_dayLimit': t_dayLimit,
                'addition': addition,
                'fixes': fixes}
    }
    
    if invest:
        maximum_wind = data['wind'][datetimeindex].max()
        maximum_demand = kwargs_all['demand'].max()
        max_used_for_normalization = max(maximum_wind, maximum_demand)
    
    # Update some kwargs since they have been changed (i.e. normalized) for investment modeling
    kwargs_all_invest = kwargs_all.copy()
    if not invest:
        kwargs_all_invest['demand'] = kwargs_all['demand'].div(kwargs_all['demand'].max())
    else:
        kwargs_all_invest['demand'] = kwargs_all['demand'].div(max_used_for_normalization)
        kwargs_all_invest['max_demand'] = None
        kwargs_all_invest['max_capacity_down'] = None
        kwargs_all_invest['max_capacity_up'] = None

    kwargs_all_invest['capacity_up'] = kwargs_all['capacity_up'].div(kwargs_all['capacity_up'].max())
    kwargs_all_invest['capacity_down'] = kwargs_all['capacity_down'].div(kwargs_all['capacity_down'].max())
    
    # Actually build the units
    if approach in ["DIW", "oemof", "DLR"]:
        
        if not invest:
            demand_dsm = solph.custom.SinkDSM(**kwargs_all_invest,
                                        **kwargs_dict[approach])
        else:
            demand_dsm = solph.custom.SinkDSM(**kwargs_all_invest,
                                        **kwargs_dict[approach],
                                        investment=solph.options.Investment(
                                             existing=existing,
                                             minimum=minimum,
                                             maximum=maximum,
                                             ep_costs=ep_costs))

A function serves to create and solve the optimization model and store the results.

In [ ]:
def solve_model(es, solver='cbc'):
    """Create and optimization model and solve it. Return the solved model and its energy system. """
    
    # Create Model
    model = solph.Model(es)

    # Solve Model
    model.solve(solver=solver, solve_kwargs={'tee': False})

    # Save Results
    es.results['main'] = solph.processing.results(model)
    es.results['meta'] = solph.processing.meta_results(model)

    return model, es

In [ ]:
# Store the results (pd.DataFrame) in a dict, indexed by approaches
approach_dict = {}

for approach in approaches:
    es = create_energy_system(data=df_data, 
                              datetimeindex=datetimeindex, 
                              invest=False, 
                              approach=approach, 
                              delay_time=4, 
                              shed_time=2, 
                              cost_dsm_up=cost_dsm_up, 
                              cost_dsm_down_shift=cost_dsm_down_shift, 
                              cost_dsm_down_shed=cost_dsm_down_shed,
                              efficiency=1, 
                              shed_eligibility=False, 
                              shift_eligibility=True)

    model, es = solve_model(es, solver=solver)

    approach_dict[approach] = extract_and_plot_results(data=df_data,
                                                       datetimeindex=datetimeindex, 
                                                       model=model,
                                                       es=es, 
                                                       approach=approach, 
                                                       case="simple", 
                                                       invest=False)

# Example 3: Model investments in demand response
This example builds up on the previous one.
* The boolean parameter invest is set to True, thus an optimized amount of investments in demand response should occur.
* Some additional attributes need to be defined for modeling investments.
* We attribute demand response with very low investment expenses, i.e. low ep_costs.
* As demand response is the cheapest option to balance residual load in that toy system, we expect the same behaviour as in the above dispatch example ... Let's check that!

In [ ]:
# Store the results (pd.DataFrame) in a dict, indexed by approaches
approach_dict = {}

for approach in approaches:
    es = create_energy_system(data=df_data, 
                              datetimeindex=datetimeindex, 
                              invest=True, 
                              approach=approach, 
                              delay_time=4, 
                              shed_time=2, 
                              cost_dsm_up=cost_dsm_up, 
                              cost_dsm_down_shift=cost_dsm_down_shift, 
                              cost_dsm_down_shed=cost_dsm_down_shed,
                              efficiency=1, 
                              shed_eligibility=False, 
                              shift_eligibility=True,
                              flex_share_up=1,
                              flex_share_down=1,
                              ep_costs=0.001,
                              existing=0,
                              minimum=0,
                              maximum=200)

    model, es = solve_model(es, solver=solver)

    approach_dict[approach] = extract_and_plot_results(data=df_data,
                                                       datetimeindex=datetimeindex, 
                                                       model=model,
                                                       es=es, 
                                                       approach=approach, 
                                                       case="simple", 
                                                       invest=True)

In [ ]:
for k, v in approach_dict.items():
    print(f"Investments in demand response for approach {k}: \t{v[1]} MW of installed capacity.")